In [ ]:
pip install tensorflow scikit-fuzzy numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 15.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Sample Data: Replace this with your actual dataset
X = np.random.rand(1000, 6)  # 6 input features like ES, SV, SPI, CPI, etc.
y = np.random.rand(1000, 1)  # 1 target variable (EV)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the ANN model
model = Sequential()
model.add(Dense(10, input_dim=6, activation='tanh'))  # Hidden layer with 10 neurons
model.add(Dense(1, activation='linear'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.1, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Save extracted features from ANN to use in ANFIS
ann_features = model.predict(X)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4413 - val_loss: 0.2063
Epoch 2/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1688 - val_loss: 0.1179
Epoch 3/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1113 - val_loss: 0.0989
Epoch 4/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1043 - val_loss: 0.0954
Epoch 5/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0922 - val_loss: 0.0949
Epoch 6/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0973 - val_loss: 0.0947
Epoch 7/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0902 - val_loss: 0.0954
Epoch 8/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0998 - val_loss: 0.0937
Epoch 9/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0907 - val_loss: 0.0929
Epoch 10/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0938 - val_loss: 0.0926
Epoch 11/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0884 - val_loss: 0.0930
Epoch 12/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0945 - val_lo

In [ ]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np

# Simulated ann_features for example; replace with actual ANN output
ann_features = np.random.rand(10, 2)  # Example with two features; change shape as needed

# Check how many features are available
if ann_features.shape[1] == 1:
    # Use just one feature (feature1)
    feature1 = ctrl.Antecedent(np.linspace(0, 1, 100), 'feature1')
    ev = ctrl.Consequent(np.linspace(0, 1, 100), 'earned_value')

    # Define membership functions (Gaussian membership functions)
    feature1['low'] = fuzz.gaussmf(feature1.universe, 0, 0.1)
    feature1['medium'] = fuzz.gaussmf(feature1.universe, 0.5, 0.1)
    feature1['high'] = fuzz.gaussmf(feature1.universe, 1, 0.1)

    ev['low'] = fuzz.gaussmf(ev.universe, 0, 0.1)
    ev['medium'] = fuzz.gaussmf(ev.universe, 0.5, 0.1)
    ev['high'] = fuzz.gaussmf(ev.universe, 1, 0.1)

    # Define fuzzy rules for one feature
    rule1 = ctrl.Rule(feature1['low'], ev['low'])
    rule2 = ctrl.Rule(feature1['medium'], ev['medium'])
    rule3 = ctrl.Rule(feature1['high'], ev['high'])

    # Build control system
    ev_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
    ev_sim = ctrl.ControlSystemSimulation(ev_ctrl)

    # Simulate for new data (using only feature1)
    ev_sim.input['feature1'] = ann_features[0][0]

    # Compute result
    ev_sim.compute()
    print("Earned Value (using one feature):", ev_sim.output['earned_value'])

elif ann_features.shape[1] >= 2:
    # Use two features (feature1 and feature2)
    feature1 = ctrl.Antecedent(np.linspace(0, 1, 100), 'feature1')
    feature2 = ctrl.Antecedent(np.linspace(0, 1, 100), 'feature2')
    ev = ctrl.Consequent(np.linspace(0, 1, 100), 'earned_value')

    # Define membership functions (Gaussian membership functions)
    feature1['low'] = fuzz.gaussmf(feature1.universe, 0, 0.1)
    feature1['medium'] = fuzz.gaussmf(feature1.universe, 0.5, 0.1)
    feature1['high'] = fuzz.gaussmf(feature1.universe, 1, 0.1)

    feature2['low'] = fuzz.gaussmf(feature2.universe, 0, 0.1)
    feature2['medium'] = fuzz.gaussmf(feature2.universe, 0.5, 0.1)
    feature2['high'] = fuzz.gaussmf(feature2.universe, 1, 0.1)

    ev['low'] = fuzz.gaussmf(ev.universe, 0, 0.1)
    ev['medium'] = fuzz.gaussmf(ev.universe, 0.5, 0.1)
    ev['high'] = fuzz.gaussmf(ev.universe, 1, 0.1)

    # Define fuzzy rules for two features
    rule1 = ctrl.Rule(feature1['low'] & feature2['low'], ev['low'])
    rule2 = ctrl.Rule(feature1['medium'] & feature2['medium'], ev['medium'])
    rule3 = ctrl.Rule(feature1['high'] & feature2['high'], ev['high'])

    # Build control system
    ev_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
    ev_sim = ctrl.ControlSystemSimulation(ev_ctrl)

    # Simulate for new data (using feature1 and feature2)
    ev_sim.input['feature1'] = ann_features[0][0]
    ev_sim.input['feature2'] = ann_features[0][1]

    # Compute result
    ev_sim.compute()
    print("Earned Value (using two features):", ev_sim.output['earned_value'])

else:
    print("Error: ann_features does not have enough dimensions.")


Earned Value (using two features): 0.4998934431278312


In [ ]:
# Evaluate Fuzzy Logic Output
def evaluate_fuzzy_output(X_test, ann_model, fuzzy_sim):
    y_pred_fuzzy = []

    for i in range(len(X_test)):
        # Get the output from the ANN model for the test data
        ann_output = ann_model.predict(X_test[i].reshape(1, -1))

        # Check if ANN output has one or two features
        if ann_output.shape[1] == 1:
            # Use only one feature for the fuzzy logic system
            fuzzy_sim.input['feature1'] = ann_output[0][0]  # ANN's first feature
        elif ann_output.shape[1] >= 2:
            # Use two features for the fuzzy logic system
            fuzzy_sim.input['feature1'] = ann_output[0][0]  # ANN's first feature
            fuzzy_sim.input['feature2'] = ann_output[0][1]  # ANN's second feature

        # Compute fuzzy output
        fuzzy_sim.compute()

        # Append the fuzzy output (earned value) to the predictions list
        y_pred_fuzzy.append(fuzzy_sim.output['earned_value'])

    return np.array(y_pred_fuzzy)

# Calculate errors
y_pred_fuzzy = evaluate_fuzzy_output(X_test, model, ev_sim)
print("Mean Absolute Error (Fuzzy):", mean_absolute_error(y_test, y_pred_fuzzy))
print("Root Mean Squared Error (Fuzzy):", np.sqrt(mean_squared_error(y_test, y_pred_fuzzy)))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━